In [1]:
!pip install torch torchvision

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.RandomCrop(32, padding=4),  # 随机裁剪
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize((0.4914, 0.4822, 0.4465),  # 标准化
                         (0.2023, 0.1994, 0.2010))
])

In [4]:
# 加载训练集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

# 加载测试集
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

# 类别名称
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:15<00:00, 10753924.99it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 卷积层和池化层
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        # 全连接层
        self.fc1 = nn.Linear(256 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
        # Dropout层
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.pool(x)

        x = nn.functional.relu(self.conv2(x))
        x = self.pool(x)

        x = nn.functional.relu(self.conv3(x))
        x = self.pool(x)

        x = x.view(-1, 256 * 4 * 4)
        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = self.dropout(nn.functional.relu(self.fc2(x)))
        x = self.fc3(x)
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = Net().to(device)
net

Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [8]:
for epoch in range(10):  # 迭代次数
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # 每100个批次打印一次
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
print('Finished Training')

[1,   100] loss: 2.014
[1,   200] loss: 1.666
[1,   300] loss: 1.511
[2,   100] loss: 1.293
[2,   200] loss: 1.239
[2,   300] loss: 1.173
[3,   100] loss: 1.074
[3,   200] loss: 1.061
[3,   300] loss: 1.023
[4,   100] loss: 0.948
[4,   200] loss: 0.946
[4,   300] loss: 0.903
[5,   100] loss: 0.881
[5,   200] loss: 0.868
[5,   300] loss: 0.862
[6,   100] loss: 0.805
[6,   200] loss: 0.816
[6,   300] loss: 0.790
[7,   100] loss: 0.792
[7,   200] loss: 0.777
[7,   300] loss: 0.753
[8,   100] loss: 0.742
[8,   200] loss: 0.743
[8,   300] loss: 0.746
[9,   100] loss: 0.706
[9,   200] loss: 0.712
[9,   300] loss: 0.729
[10,   100] loss: 0.675
[10,   200] loss: 0.697
[10,   300] loss: 0.699
Finished Training


In [9]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('在10000张测试图片上的准确率为: %d %%' % (
    100 * correct / total))

在10000张测试图片上的准确率为: 74 %


In [10]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
for i in range(10):
    print('类别 %5s 的准确率: %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

类别 plane 的准确率: 80 %
类别   car 的准确率: 92 %
类别  bird 的准确率: 67 %
类别   cat 的准确率: 56 %
类别  deer 的准确率: 83 %
类别   dog 的准确率: 60 %
类别  frog 的准确率: 78 %
类别 horse 的准确率: 74 %
类别  ship 的准确率: 84 %
类别 truck 的准确率: 72 %
